In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv('quora_duplicate_questions.tsv', delimiter="\t")

In [3]:
dataset.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
for i in range(len(dataset.columns.values)):
    print (i, dataset.columns.values[i], end = " ")
    print()

0 id 
1 qid1 
2 qid2 
3 question1 
4 question2 
5 is_duplicate 


In [5]:
X_temp = dataset.iloc[:, 3:5].values
y_temp = dataset.iloc[: , 5].values

In [6]:
len(X_temp[0][0])

66

In [28]:
X = []
y = []

In [29]:
question_length_threshold = 100

In [30]:
for i in range(len(X_temp)):
    _temp_X = []
    _temp_y = []
    flag=False    
    if type(X_temp[i][0]) == str and type(X_temp[i][1]) == str and len(X_temp[i][0]) <= question_length_threshold and len(X_temp[i][1]) <= question_length_threshold:
        _temp_X.append(X_temp[i][0])
        _temp_X.append(X_temp[i][1])
        _temp_y.append(y_temp[i])
        flag=True
    if flag==True:
        X.append(_temp_X)
        y.append(_temp_y[0])

In [31]:
X = np.array(X)
y = np.array(y)

In [32]:
X_temp.shape

(404290, 2)

In [33]:
X.shape

(339422, 2)

In [13]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [34]:
print (type(X))

<class 'numpy.ndarray'>


In [35]:
s = set()
for i in X:
    for j in i:
        temp = j.split(" ")
        for k in temp:
            s.add(k)

In [36]:
len(s)

169706

In [37]:
vocab_size = len(s)

In [38]:
question_1 = []
question_2 = []

In [39]:
print (X[0][0])

What is the step by step guide to invest in share market in india?


In [40]:
for i in range(len(X)):
    question_1.append(X[i][0])
    question_2.append(X[i][1])

In [41]:
encoded_question_1 = [one_hot(d, vocab_size) for d in question_1]
encoded_question_2 = [one_hot(d, vocab_size) for d in question_2]

In [42]:
max_length = -1
for i in range(len(encoded_question_1)):
    max_length = max(max_length, len(encoded_question_1[i]), len(encoded_question_2[i]))

In [43]:
max_length

31

In [44]:
padded_question_1 = pad_sequences(encoded_question_1, maxlen=max_length, padding='post')
padded_question_2 = pad_sequences(encoded_question_2, maxlen=max_length, padding='post')

In [45]:
padded_question = []
for i in range(len(padded_question_1)):
    padded_question.append(list(padded_question_2[i]) + list(padded_question_1[i]))

In [46]:
padded_question = np.array(padded_question)

In [47]:
padded_question.shape

(339422, 62)

In [48]:
y.shape

(339422,)

In [49]:
X_final = padded_question

# Training

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size = 0.2, random_state = 0)

In [51]:
model = Sequential()
model.add(Embedding(vocab_size, 30, input_length=max_length*2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 62, 30)            5091180   
_________________________________________________________________
flatten_1 (Flatten)          (None, 1860)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               930500    
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
_________________________________________________________________
dense_3 (Dense)              (None, 500)               250500    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 501       
Total params: 6,523,181
Trainable params: 6,523,181
Non-trainable params: 0
_________________________________________________________________


In [116]:
model.fit(X_train, y_train, epochs=3)

Epoch 1/3
271537/271537 [==============================] - 432s 2ms/step - loss: 0.1250 - acc: 0.9507
Epoch 2/3
271537/271537 [==============================] - 437s 2ms/step - loss: 0.0900 - acc: 0.9651 0s - loss: 0.0899 - acc
Epoch 3/3
271537/271537 [==============================] - 434s 2ms/step - loss: 0.0688 - acc: 0.9739


In [117]:
score = model.evaluate(X_test, y_test)

67885/67885 [==============================] - 35s 509us/step


In [118]:
print(score[1])

0.774117993668


In [142]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


# Testing

In [ ]:
from keras.models import model_from_json
from keras.optimizers import SGD
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
score = model.evaluate(X_test, y_test)
print(score[1])

In [61]:
question1 = ["How to become a good doctor?"]
question2 = ["What i have to do for being a good doctor?"]

In [62]:
encoded_question_test_1 = [one_hot(d, vocab_size) for d in question1]
encoded_question_test_2 = [one_hot(d, vocab_size) for d in question2]

In [63]:
padded_question_test_1 = pad_sequences(encoded_question_test_1, maxlen=max_length, padding='post')
padded_question_test_2 = pad_sequences(encoded_question_test_2, maxlen=max_length, padding='post')

In [64]:
padded_question_test = []
for i in range(len(padded_question_test_1)):
    padded_question_test.append(list(padded_question_test_2[i]) + list(padded_question_test_1[i]))

In [65]:
padded_question_test = np.array(padded_question_test)

In [66]:
y = loaded_model.predict(padded_question_test)

In [67]:
print("The two statements are",y[0][0] * 100, "% similar", end = " ")

The two statements are 94.8739111423 % similar 